<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Masterproject_final_tensor_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision.transforms.functional as TF 

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os
import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
from torch import optim
from torch.optim import lr_scheduler
from functools import partial
import matplotlib.pyplot as plt
!pip install partitura
import partitura

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 218 kB 16.9 MB/s 
     |████████████████████████████████| 334 kB 56.6 MB/s 
     |████████████████████████████████| 103 kB 55.6 MB/s 
     |████████████████████████████████| 51 kB 8.5 MB/s 
     |████████████████████████████████| 181 kB 50.7 MB/s 


In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 5461, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 5461 (delta 28), reused 59 (delta 5), pack-reused 5364
Receiving objects: 100% (5461/5461), 6.25 MiB | 9.29 MiB/s, done.
Resolving deltas: 100% (4627/4627), done.
Checking out files: 100% (5329/5329), done.


# Dataloader

In [3]:
batch_size = 1 
PATH_TO_DATA = "AI-MA_project/bach_pr_fugues"
#PATH_TO_DATA = "AI-MA_project/pianoroll_88"
workers = 0

In [4]:
class MusicDataset(Dataset):

    def __init__(self, data_dir, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
        self.labels = labels
        self.pr_dict = {}
        len_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: ### normal sollte es egal sein wenn voice_4 bei manchen nicht existiert - wenn nicht condition einführen damit das funktioniert
                        loaded_obj = pickle.load(f)  
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))
                        
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
    
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl"))) 
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)     
                        voice_files.append(loaded_obj)

                self.pr_dict[self.labels[iLabel]] = voice_files 


    def __len__(self):
      
        return len(self.pr_dict[self.labels[0]])
  

    def __getitem__(self, idx):          

        out_list = []
        for key, value in self.pr_dict.items():
            out_list.append(self.pr_dict[key][idx])    

        v0 = torch.tensor(out_list[0].T)
        v1 = torch.tensor(out_list[1].T)
        v2 = torch.tensor(out_list[2].T)
        v3 = torch.tensor(out_list[3].T)
        v_all = torch.tensor(out_list[4].T) 
        length = self.pr_dict["length"][idx]


        return (v0, v1, v2, v3, v_all, length)

In [98]:
class MusicDataset_new(Dataset):

    def __init__(self, data_dir, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        self.name_list = ["1f01","1f02","1f03","1f04","1f05","1f06","1f07","1f08","1f09","1f10","1f11","1f12","1f13","1f14","1f15","1f16","1f17","1f18","1f19","1f20","1f21","1f22","1f23","1f24","2f01","2f02","2f03","2f04","2f05","2f06","2f07","2f08","2f09","2f10","2f11","2f12","2f13","2f14","2f15","2f16","2f17","2f18","2f19","2f20","2f21","2f22","2f23","2f24"]
        self.name_list_voice_3 =  ['1f01', '1f05', '1f12', '1f14', '1f16', '1f17', '1f18', '1f23', '1f24', '2f02', '2f05', '2f07', '2f08', '2f09', '2f16', '2f17',  '2f22', '2f23']
        labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
        self.labels = labels
        self.pr_dict = {}
        len_list = []
        nbr_voices_list = []
        file_names_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: ### normal sollte es egal sein wenn voice_4 bei manchen nicht existiert - wenn nicht condition einführen damit das funktioniert
                        loaded_obj = pickle.load(f)  
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))

                        file_names_list.append(name[-8:-4])

                    if "AI-MA_project/bach_pr_fugues/voice_3/voice_3_" + name[49:53] + ".pkl" in sorted(glob.glob(os.path.join(PATH_TO_DATA, "voice_3", "*.pkl"))):
                        nbr_voices_list.append(4)
                    else:
                        nbr_voices_list.append(3)
                        
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
                self.pr_dict["nbr_voices"] = nbr_voices_list
                self.pr_dict["name"] = file_names_list
                #print(self.pr_dict["name"])


            if iLabel == 3:  
                voice_files = []
                file_names_3 = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl"))) 
                file_names_2 = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[2], "*.pkl")))   
                
                ###### loop over all filnames in voices_2 and if an element there is not present in voices_3: append "missing" to the voice_files of label=3 => important bc. self.pr_dict[voice_3] has then len 42 and otherwise it would only have len 18  .. these "missing" el are not considered later in the dataloader (if len=3 is a diff case of get_idx)
                for name in file_names_2:
                    if name[45:49] in self.name_list_voice_3:
                      correct_name_3 = "AI-MA_project/bach_pr_fugues/voice_3/voice_3_" + name[45:49] + ".pkl"
                      with open(correct_name_3 ,'rb') as f:  
                            loaded_obj = pickle.load(f)  
                            voice_files.append(loaded_obj)
                    else:
                      voice_files.append("missing")

                self.pr_dict[self.labels[iLabel]] = voice_files 
                

                
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl"))) 
                for name in file_names:
                    with open(name ,'rb') as f: 
                          loaded_obj = pickle.load(f)     
                          voice_files.append(loaded_obj)

                self.pr_dict[self.labels[iLabel]] = voice_files 


    def __len__(self):
        return len(self.pr_dict[self.labels[0]])

    def __getitem__(self, idx):      
        out_list = []
        
        if self.pr_dict["nbr_voices"][idx] == 4:
            for key, value in self.pr_dict.items():
              out_list.append(self.pr_dict[key][idx])
                              
            v0 = torch.tensor(out_list[0].T)
            v1 = torch.tensor(out_list[1].T)
            v2 = torch.tensor(out_list[2].T)
            v3 = torch.tensor(out_list[3].T)
            v_all = torch.tensor(out_list[4].T) 
            length = self.pr_dict["length"][idx]
            nbr_voices = self.pr_dict["nbr_voices"][idx]
            file_name = self.pr_dict["name"][idx]

            voices = torch.stack([v0, v1, v2, v3, v_all], dim=2)
            
            return (voices, length, nbr_voices, file_name)
        
        if self.pr_dict["nbr_voices"][idx] == 3:

            for key, value in self.pr_dict.items():
                if key != "voice_3":
                  out_list.append(self.pr_dict[key][idx]) 
            
            v0 = torch.tensor(out_list[0].T)
            v1 = torch.tensor(out_list[1].T)
            v2 = torch.tensor(out_list[2].T)
            v3 = torch.zeros(v2.shape)
            v_all = torch.tensor(out_list[3].T) 
            length = self.pr_dict["length"][idx]
            nbr_voices = self.pr_dict["nbr_voices"][idx]
            file_name = self.pr_dict["name"][idx]
            
            voices = torch.stack([v0, v1, v2, v3, v_all], dim=2)

            
            return (voices, length, nbr_voices, file_name)
        

In [99]:
#dataset = MusicDataset(PATH_TO_DATA)
dataset = MusicDataset_new(PATH_TO_DATA)
loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

In [101]:
"""
for i, sample_batched in enumerate(loader):
    all_voices, length, nbr_voices = sample_batched
    if nbr_voices ==3:
      print(i,nbr_voices,all_voices.shape)
    else:
      print(i,nbr_voices)
"""
for i, sample_batched in enumerate(loader):
    all_voices, length, nbr_voices, file_name = sample_batched
    print(file_name[0])


1f01
1f03
1f05
1f06
1f07
1f08
1f09
1f11
1f12
1f13
1f14
1f16
1f17
1f18
1f19
1f21
1f23
1f24
2f01
2f02
2f03
2f04
2f05
2f06
2f07
2f08
2f09
2f11
2f12
2f13
2f14
2f15
2f16
2f17
2f18
2f19
2f20
2f21
2f22
2f23
2f24


In [8]:
"""
for i, sample_batched in enumerate(loader):
    all_voices, length, nbr_voices = sample_batched
    if nbr_voices ==3:
      print(i,nbr_voices,all_voices.shape)
    else:
      print(i,nbr_voices)
"""
for i, sample_batched in enumerate(loader):
  if i ==10:
    all_voices, length, nbr_voices, _ = sample_batched
    all_voices_pr = all_voices[0,:,:,-1].numpy()
    
    note_array = partitura.utils.pianoroll_to_notearray(all_voices[0,:,:,-1].numpy(), time_div=12, time_unit='beat')
    print(note_array.shape)
    print(note_array[:10])
    print(note_array.dtype.names)

    #print(i,nbr_voices,all_voices.shape)

(2868,)
[(0, 2.75, 0.08333334, 1) (1, 2.75, 0.08333334, 1)
 (2, 2.75, 0.08333334, 1) (3, 2.75, 0.08333334, 1)
 (4, 2.75, 0.08333334, 1) (5, 2.75, 0.08333334, 1)
 (6, 2.75, 0.08333334, 1) (7, 2.75, 0.08333334, 1)
 (8, 2.75, 0.08333334, 1) (9, 2.75, 0.08333334, 1)]
('pitch', 'onset_beat', 'duration_beat', 'velocity')


# Define Music - Model


Define UNET 

In [9]:
class UNET(nn.Module):
    
    def __init__(self, in_channels=1, classes=1):
        super(UNET, self).__init__()
        self.layers = [in_channels, 64, 128, 256, 512, 1024]
        
        self.double_conv_downs = nn.ModuleList([self.__double_conv(layer, layer_n) for layer, layer_n in zip(self.layers[:-1], self.layers[1:])])
        
        self.up_trans = nn.ModuleList([nn.ConvTranspose2d(layer, layer_n, kernel_size=2, stride=2) for layer, layer_n in zip(self.layers[::-1][:-2], self.layers[::-1][1:-1])])
            
        self.double_conv_ups = nn.ModuleList([self.__double_conv(layer, layer//2) for layer in self.layers[::-1][:-2]])
        
        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.final_conv = nn.Conv2d(64, classes, kernel_size=1)

        
    def __double_conv(self, in_channels, out_channels):
        conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        return conv
    
    def forward(self, x):
        # down layers
        concat_layers = []
        
        for down in self.double_conv_downs:
            x = down(x)
            if down != self.double_conv_downs[-1]:
                concat_layers.append(x)
                x = self.max_pool_2x2(x)
        
        concat_layers = concat_layers[::-1]
        
        # up layers
        for up_trans, double_conv_up, concat_layer  in zip(self.up_trans, self.double_conv_ups, concat_layers):
            x = up_trans(x)
            if x.shape != concat_layer.shape:
                x = TF.resize(x, concat_layer.shape[2:])
            
            concatenated = torch.cat((concat_layer, x), dim=1)
            x = double_conv_up(concatenated)
            
        x = self.final_conv(x)
        
        return x 

In [10]:
class MusicNetwork(nn.Module):

    def __init__(self, network_type,output_dim=88, hidden_dim=300, rnn_depth=1, cell_type="GRU"):                 
        super(MusicNetwork, self).__init__()

        self.network_type = network_type
        self.n_out = output_dim
        input_dim = output_dim 
        rnn_cell = nn.GRU
        self.rnn = rnn_cell(input_size=input_dim, hidden_size=hidden_dim, num_layers=rnn_depth, batch_first=True)
        self.cnn = UNET(in_channels=1, classes=4)
        self.top_layer_voice_0 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_1 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_2 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_3 = nn.Linear(hidden_dim, self.n_out)
        self.loss = nn.CrossEntropyLoss(reduction="mean")

    def compute_outputs(self, sentences, sentences_len):
        if self.network_type == "RNN":
          rnn_out ,_= self.rnn(sentences)     
          out_0 = self.top_layer_voice_0(rnn_out)
          out_1 = self.top_layer_voice_1(rnn_out)
          out_2 = self.top_layer_voice_2(rnn_out)
          out_3 = self.top_layer_voice_3(rnn_out)
          

          return torch.stack([out_0, out_1, out_2, out_3], dim=1)

        else: 
          sentences = sentences[:,None]
          out = self.cnn(sentences)
          return out                                                            
        

    def forward(self, voices, sentences_len, nbr_voices):            

        # Compute the outputs. The shape is (max_len, n_sentences, n_labels).
        scores_comb = self.compute_outputs(voices[:,:,:,-1], sentences_len)

        # Flatten the outputs and the labels, to compute the loss.
        # The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
        score_0  = scores_comb[:,0,:,:].view(-1, self.n_out)
        score_1  = scores_comb[:,1,:,:].view(-1, self.n_out)
        score_2  = scores_comb[:,2,:,:].view(-1, self.n_out)
        score_3  = scores_comb[:,3,:,:].view(-1, self.n_out)

        v0 = voices[:,:,:,0].squeeze()
        v1 = voices[:,:,:,1].squeeze()
        v2 = voices[:,:,:,2].squeeze()
        v3 = voices[:,:,:,3].squeeze()

        if nbr_voices==4:
            loss = self.loss(score_0, v0) + self.loss(score_1, v1) + self.loss(score_2, v2) + self.loss(score_3, v3)           
        else:
            loss = self.loss(score_0, v0) + self.loss(score_1, v1) + self.loss(score_2, v2) 
        
        return loss
        


    def predict(self, sentences, sentences_len,monophonic=True):

        # Compute the outputs from the linear units.
        scores_comb = self.compute_outputs(sentences, sentences_len)

        if monophonic==False:
            sum = scores_comb * sentences[:,None,:,:]
            return np.squeeze(sum.cpu().numpy())
            

        else:
            # Select the top-scoring labels. The shape is now (max_len, n_sentences).
            #print("predictmethod",scores_comb.shape)
            predicted = scores_comb.argmax(dim=3)
            return np.squeeze(predicted.cpu().numpy())

In [11]:
"""
network_type= "RNN"
monophonic = True
his = start_experiment(2, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, network_type, learn_all)
"""

'\nnetwork_type= "RNN"\nmonophonic = True\nhis = start_experiment(2, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, network_type, learn_all)\n'

# Define Training Process

In [12]:
def train(epochs, lr, hidden_dim, momentum, rnn_depth, device, rnn_cell, weight_decay,network_type, train_dataloader, val_dataloader=None):
    
    output_dim = 88
    model = MusicNetwork(network_type, output_dim, hidden_dim, rnn_depth, cell_type)              
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = lr_scheduler.MultiStepLR(optimizer, [epochs // 2], gamma=0.1, verbose=True)

    history = training_loop(model, optimizer, train_dataloader,monophonic, epochs=epochs, val_dataloader=val_dataloader, device=device, scheduler=scheduler)

    return model, history

In [13]:
def training_loop(model,optimizer, train_dataloader, monophonic, epochs=50, val_dataloader=None, device=None, scheduler=None):
    if device is None:
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f"Training on device: {device}")

    print("monophonic set to:",monophonic)
    model = model.to(device)
    history = defaultdict(list)

    for i_epoch in range(1, epochs + 1):
        loss_sum = 0

        accuracy_sum_list = [0 for i in range(5)]                                   ########## FIXED FOR 5 voices MAX right now - b.c. FUGUES have max 5 voices
        val_accuracy_sum_list = [0 for i in range(5)]                               ########## FIXED FOR 5 voices MAX right now - b.c. FUGUES have max 5 voices

        accuracy_v_all_sum = 0
        model.train()
        accuracy_sum = 0
        

        for idx, (voices, lens, nbr_voices, _) in enumerate(train_dataloader):  
            
            voices = voices.to(device).float()
            optimizer.zero_grad()
            loss = model.forward(voices, lens, nbr_voices)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()    

            if monophonic == False:
                with torch.no_grad():
                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)                      

                    v_pred_argm = torch.tensor(np.argmax(prediction,axis=0))
                    mask_pred = (prediction.sum(axis=0) == 0)
                    v_pred_argm[mask_pred] = -1
                    v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)
                    
                    single_voices = voices[:,:,:,:-1]

                    v_ori_argm = torch.argmax(np.squeeze(single_voices,axis=0).cpu(),axis=2)
                    mask_ori = ((np.squeeze(single_voices,axis=0).cpu()).sum(axis=2) == 0).numpy()
                    v_ori_argm[mask_ori] = -1
                    v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)
                    acc = accuracy_score(v_pred_flat,v_ori_flat)  
                    accuracy_sum += acc 


            if monophonic == True:
                with torch.no_grad():

                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                    prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)

                    truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()

                    acc_list = [0 for i in range(len(prediction[0,:]))]

                    for i in range(len(prediction[0,:])):
                      acc_list[i] = accuracy_score(prediction[:,i], truth[:,i])
                      accuracy_sum_list[i] += acc_list[i]/len(lens)

                    """
                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                    prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)
                    truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()
                    comp_list = [0,0,0,0]
                    idx_list = [-1,-1,-1,-1]
                    voices_v = [0,0,0,0]

                    for i in range(4):
                      for j in range(4):
                        intermed = sum(prediction[:,i] == truth[:,j])
                        if intermed > comp_list[i] :
                          comp_list[i] = intermed
                          idx_list[i] = j
                          voices_v[i] = intermed / len(prediction[:,i])
                    avc = 100 * sum(voices_v) / 4
                    """

        train_loss = loss_sum / len(train_dataloader)

        # normalize according to the number of batches
        if monophonic == True:

            train_acc_list = np.array(accuracy_sum_list) / len(train_dataloader)
            train_acc_list[3] = accuracy_sum_list[3] / 18                        ## bc only 18 pieces with len 3
            train_acc_list[4] = accuracy_sum_list[4] / 2                         ##### CHECK IF 2 or 3 pieces with len 4

            history["train_loss"].append(train_loss)
            history["train_acc"].append(train_acc_list)
            print("Train Loss: {}, Train Accuracy_0 : {}, Train Accuracy_1 : {},Train Accuracy_2 : {}, Train Accuracy_3 : {}, Train Accuracy_4 : {}".format(train_loss, train_acc_list[0], train_acc_list[1], train_acc_list[2], train_acc_list[3],train_acc_list[4])) 


        if monophonic == False:
            train_accuracy = accuracy_sum / len(train_dataloader)

            history["train_loss"].append(train_loss)
            history["train_accuracy"].append(train_accuracy)
            print("Train Loss: {}, Train Accuracy : {}".format(train_loss, train_accuracy)) 


        if monophonic == True:
            if val_dataloader is not None:
                # Evaluate on the validation set
                model.eval()
                with torch.no_grad():

                    for voices, lens, nbr_voices, _ in val_dataloader:

                        voices = voices.to(device).float()

                        prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                        prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)

                        truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()

                        acc_list = [0 for i in range(len(prediction[0,:]))]

                        for i in range(len(prediction[0,:])):
                          acc_list[i] = accuracy_score(prediction[:,i], truth[:,i])
                          val_accuracy_sum_list[i] += acc_list[i]/len(lens)

                    val_acc_list = np.array(val_accuracy_sum_list) / len(val_dataloader)
                    val_acc_list[3] = val_accuracy_sum_list[3] / 18                         ## bc only 18 pieces with len 3
                    val_acc_list[4] = val_accuracy_sum_list[4] / 2                          ##### CHECK IF 2 or 3 pieces with len 4

                history["val_acc"].append(val_acc_list)
                print(" Validation Accuracy_0 : {}, Validation Accuracy_1 : {}, Validation Accuracy_2 : {}, Validation Accuracy_3 : {}, Validation Accuracy_4 : {}".format(val_acc_list[0], val_acc_list[1], val_acc_list[2], val_acc_list[3],val_acc_list[4]))


        if monophonic == False:
            if val_dataloader is not None:
                # Evaluate on the validation set
                model.eval()
                accuracy_sum = 0
                
                with torch.no_grad():
                    for voices, lens, nbr_voices, _ in val_dataloader:

                        voices = voices.to(device).float()
                        
                        # Predict the model's output on a batch
                        prediction = model.predict(voices[:,:,:,-1], lens, monophonic)                      

                        v_pred_argm = torch.tensor(np.argmax(prediction,axis=0))
                        mask_pred = (prediction.sum(axis=0) == 0)
                        v_pred_argm[mask_pred] = -1
                        v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)
                        
                        single_voices = voices[:,:,:,:-1]

                        v_ori_argm = torch.argmax(np.squeeze(single_voices,axis=0).cpu(),axis=2)
                        mask_ori = ((np.squeeze(single_voices,axis=0).cpu()).sum(axis=2) == 0).numpy()
                        v_ori_argm[mask_ori] = -1
                        v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)
                        acc = accuracy_score(v_pred_flat,v_ori_flat)  
                        accuracy_sum += acc 
                        
                    # normalize according to the number of batches
                    val_accuracy = accuracy_sum / len(val_dataloader)

                history["val_accuracy"].append(val_accuracy)  
                print(" Validation Accuracy : {}".format(val_accuracy))


        if scheduler is not None:
            scheduler.step()

    # save the model
    #torch.save(model, Path("./AI-MA_project/model_temp_epoch{}.pkl".format(i_epoch)))
    torch.save({'model_state_dict': model.state_dict()}, Path("./AI-MA_project/model_temp_epoch{}.pkl".format(i_epoch)))

    return history

In [14]:
#network_type= "RNN"
#monophonic = True
#his = start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, network_type, learn_all)

In [15]:
"""
network_type= ["CNN","RNN"]
monophonic_list = [True,False]

for net in network_type:
    for monophonic in monophonic_list: 
        print("network set to:",net,"monophnic:",monophonic)
        start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, net, learn_all)
"""

'\nnetwork_type= ["CNN","RNN"]\nmonophonic_list = [True,False]\n\nfor net in network_type:\n    for monophonic in monophonic_list: \n        print("network set to:",net,"monophnic:",monophonic)\n        start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, net, learn_all)\n'

In [16]:
def start_experiment( epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell, decay,network_type, learn_all):
    
    trainer = partial(train,epochs, lr, hidden_dim, momentum, rnn_depth, device, cell, decay, network_type)

    if learn_all == True:
        print("Learning from full dataset")
        train_dataset = MusicDataset_new(PATH_TO_DATA) #MusicDataset(PATH_TO_DATA)
        train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)
                
        _, history = trainer(train_dataloader)

    
    else:
        # Divide train and validation set
        path_train, path_validation = sklearn.model_selection.train_test_split(PATH_TO_DATA, test_size=0.15, random_state=10,)

        print("Train and validation lenghts: ", len(path_train), len(path_validation))
        train_dataset = MusicDataset_new(PATH_TO_DATA) #MusicDataset(PATH_TO_DATA)
        validation_dataset = MusicDataset_new(path_validation) #MusicDataset(path_validation)
        
        train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)
        val_dataloader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

        _, history = trainer(train_dataloader, val_dataloader)

    return history

# Hyperparameter choice

In [17]:
model = MusicNetwork
epochs = 1
lr = 0.001  
momentum = 0.9
decay = 1e-4
hidden_dim = 300
bs = 1
rnn_depth = 2 
device = None                 #if None:  choses device automatically
cell_type = "GRU"
optimizer = "Adam"
learn_all = "False"           # False -> uses train + valid set
network_type= "CNN"

monophonic = True

# Run the Experiment

In [18]:
his = start_experiment(epochs, lr, hidden_dim, bs, momentum, rnn_depth, device, cell_type, decay, network_type, learn_all)

Train and validation lenghts:  23 5
Adjusting learning rate of group 0 to 1.0000e-04.
Training on device: cuda
monophonic set to: True
Train Loss: 5.750662041873467, Train Accuracy_0 : 0.5686166311008525, Train Accuracy_1 : 0.4645494709600349,Train Accuracy_2 : 0.40493887404482254, Train Accuracy_3 : 0.4454940077274219, Train Accuracy_4 : 0.0
 Validation Accuracy_0 : 0.8004269677549892, Validation Accuracy_1 : 0.5932059738189432, Validation Accuracy_2 : 0.566492645521635, Validation Accuracy_3 : 0.6599445222807715, Validation Accuracy_4 : 0.0
Adjusting learning rate of group 0 to 1.0000e-04.


# Accuracy evalutaion F-scores

1. create folder with part object of all pieces 
2. load a piece from dataloader with true labels, the mixed piece and the part object 
3. create notearray from part object
4. take 1 note from notearrray - input -> find corresponding frame by looking at time and pitch


Output: pianoroll

1 note in notearray could be mulitple bins

take 1 note from notearrray - input -> find corresponding frame by looking at time and pitch

note start at same time with different pitch -> different notes

for each note array find corresponding matrix -> 


if note is only composed by 1 bin: save indx of vocie -> save it to note array

if more than 1: look what are idx that compose this note -> majority note -> save it for the note array (if its 50/50 take it random -> count how often this happens) 


with idx : in note_array find which note corresponds to what voice

In [19]:
train_dataset = MusicDataset_new(PATH_TO_DATA) #MusicDataset(PATH_TO_DATA)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

In [ ]:
output_dim = 88
model = MusicNetwork(network_type, output_dim, hidden_dim, rnn_depth, cell_type)  
checkpoint = torch.load("./AI-MA_project/model_temp_epoch1.pkl")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Create dic with key:filename, val: part_obj

In [ ]:
path_parts = "AI-MA_project/bach_fugues"
part_dic = {}

#### create a list with all filenames in the right order ####
file_names_part = []
for filename in sorted(os.listdir(path_parts)):
    if not filename.endswith('.mid'): continue
    file_names_part.append(filename[3:7])
#print(file_names_part)

#### create a list with all part objects in the right order ####
part_list = []
for filename in sorted(os.listdir(path_parts)):
    if not filename.endswith('.mid'): continue
    fullname = os.path.join(path_parts, filename)
    part = partitura.load_score_midi(fullname)
    part_list.append(part)
#print(part_list)

#### create a dict with keys:filenames , values: part object ####
for i in range(len(file_names_part)):
      part_dic[file_names_part[i]] = part_list[i]

In [80]:
part_dic.keys(),part_dic.values()

(dict_keys(['1f01', '1f02', '1f03', '1f04', '1f05', '1f06', '1f07', '1f08', '1f09', '1f10', '1f11', '1f12', '1f13', '1f14', '1f15', '1f16', '1f17', '1f18', '1f19', '1f20', '1f21', '1f22', '1f23', '1f24', '2f01', '2f02', '2f03', '2f04', '2f05', '2f06', '2f07', '2f08', '2f09', '2f10', '2f11', '2f12', '2f13', '2f14', '2f15', '2f16', '2f17', '2f18', '2f19', '2f20', '2f21', '2f22', '2f23', '2f24']),
 dict_values([[<partitura.score.Part object at 0x7fa7795a69d0>, <partitura.score.Part object at 0x7fa779543f90>, <partitura.score.Part object at 0x7fa779494b50>, <partitura.score.Part object at 0x7fa7793fa490>], [<partitura.score.Part object at 0x7fa779673390>, <partitura.score.Part object at 0x7fa7791c2110>, <partitura.score.Part object at 0x7fa779141590>], [<partitura.score.Part object at 0x7fa778d28a90>, <partitura.score.Part object at 0x7fa778caf410>, <partitura.score.Part object at 0x7fa778ad4050>], [<partitura.score.Part object at 0x7fa778812ad0>, <partitura.score.Part object at 0x7fa77883

geht gerade nur für monophonic True

In [152]:
accuracy_sum_list = [0 for i in range(5)]

def evaluate_accuracy(model, train_dataloader, part_dic):
    #print("part_dic:",part_dic)



    for idx, (voices, lens, nbr_voices, file_name) in enumerate(train_dataloader):
            #check if elements match
            if nbr_voices[0]!=len(part_dic[file_name[0]]):
              print("ERROR: nbr_voices from part DOES NOT MATCH data loader:" ) 
            
            # load correct part object
            file_name = file_name[0]

            part = part_dic[file_name]


            part_zero = part[0]
            part_one = part[1]
            part_two = part[2]

            if len(part) ==4:
              part_three = part[3]
              note_array_3 = part_three.note_array

            note_array_0 = part_zero.note_array
            note_array_1 = part_one.note_array
            note_array_2 = part_two.note_array
            
            
            print("note_array shapes for filename:",file_name)
            print(note_array_0.shape)
            print(note_array_1.shape)
            print(note_array_2.shape)
            print(note_array_3.shape)

            print(note_array_0[:5])
            print(note_array_0.dtype.names)
            
            print(note_array_1[:5])
            print(note_array_1.dtype.names)

            print(note_array_2[:5])
            print(note_array_2.dtype.names)

            print(note_array_3[:5])
            print(note_array_3.dtype.names)



            
            # do model prediction
            model.eval()
            voices = voices.to(device).float()
            monophonic=True
            with torch.no_grad():

                prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)

                truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()


                
                acc_list = [0 for i in range(len(prediction[0,:]))]
                
                
                for i in range(len(prediction[0,:])):
                  acc_list[i] = accuracy_score(prediction[:,i], truth[:,i])
                  accuracy_sum_list[i] += acc_list[i]/len(lens)
                
                #print(accuracy_sum_list)
            #print(file_name)

    
    train_acc_list = np.array(accuracy_sum_list) / len(train_dataloader)
    train_acc_list[3] = accuracy_sum_list[3] / 18                               ## bc only 18 pieces with len 3
    train_acc_list[4] = accuracy_sum_list[4] / 2                                ##### CHECK IF 2 or 3 pieces with len 4


    return train_acc_list

In [153]:
evaluate_accuracy(model,train_dataloader,part_dic)

note_array shapes for filename: 1f01
(217,)
(202,)
(176,)
(143,)
[(6.  , 0.5  , 6.  , 0.5  , 720, 60, 67, 1, 'n0')
 (6.5 , 0.5  , 6.5 , 0.5  , 780, 60, 69, 1, 'n1')
 (7.  , 0.5  , 7.  , 0.5  , 840, 60, 71, 1, 'n2')
 (7.5 , 0.75 , 7.5 , 0.75 , 900, 90, 72, 1, 'n3')
 (8.25, 0.125, 8.25, 0.125, 990, 15, 74, 1, 'n4')]
('onset_beat', 'duration_beat', 'onset_quarter', 'duration_quarter', 'onset_div', 'duration_div', 'pitch', 'voice', 'id')
[(0.  , 0.5  , 0.  , 0.5  ,   0, 60, 60, 1, 'n217')
 (0.5 , 0.5  , 0.5 , 0.5  ,  60, 60, 62, 1, 'n218')
 (1.  , 0.5  , 1.  , 0.5  , 120, 60, 64, 1, 'n219')
 (1.5 , 0.75 , 1.5 , 0.75 , 180, 90, 65, 1, 'n220')
 (2.25, 0.125, 2.25, 0.125, 270, 15, 67, 1, 'n221')]
('onset_beat', 'duration_beat', 'onset_quarter', 'duration_quarter', 'onset_div', 'duration_div', 'pitch', 'voice', 'id')
[(12.  , 0.5  , 12.  , 0.5  , 1440, 60, 55, 1, 'n419')
 (12.5 , 0.5  , 12.5 , 0.5  , 1500, 60, 57, 1, 'n420')
 (13.  , 0.5  , 13.  , 0.5  , 1560, 60, 59, 1, 'n421')
 (13.5 , 0.75 

KeyboardInterrupt: ignored

In [232]:
accuracy_sum_list = [0 for i in range(5)]

def evaluate_accuracy_for_one(model, train_dataloader, part_dic):
    #print("part_dic:",part_dic)



    for idx, (voices, lens, nbr_voices, file_name) in enumerate(train_dataloader):
            #check if elements match
            if idx == 0:
                if nbr_voices[0]!=len(part_dic[file_name[0]]):
                  print("ERROR: nbr_voices from part DOES NOT MATCH data loader:" ) 
                
                # load correct part object
                file_name = file_name[0]

                part = part_dic[file_name]

                part_zero = part[0]
                note_array_0 = part_zero.note_array
                
                #print("note array 0 shape:", note_array_0.shape)
                #print(note_array_0)#[:5])
                #print(note_array_0.dtype.names)

                onset_beat = note_array_0["onset_beat"]
                duration_beat = note_array_0["duration_beat"]

                #print("onset_beat:", onset_beat)
                #print("duration_beat:", duration_beat)

                note_idx = [onset_beat[0]*12]

                for i in range(len(onset_beat)):
                    onset = onset_beat[i]
                    duration = duration_beat[i]
                  
                    note_range = 12 * (onset+duration)
                    note_idx.append(note_range)

                print("note_idx:",note_idx)

                onset_beat_0 = partitura.utils.note_array_from_part(part_zero)['onset_beat'][-1]
                duration_beat_0 = partitura.utils.note_array_from_part(part_zero)['duration_beat'][-1]
                beat_0 = onset_beat_0 + duration_beat_0

                #pr_zero, idx = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12,return_idxs=True,piano_range=True)
                #print("Pr:",pr_zero)
                #print("idx",idx)
                #pr_zero = pr_zero.toarray()
                #print("pr_zero:",pr_zero.shape)
                               
                # do model prediction
                model.eval()
                voices = voices.to(device).float()
                monophonic=True
                with torch.no_grad():
                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                    prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)
                    truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()


                #print("prediction shape:",prediction.shape)
                print("prediction 0:", prediction[0:72,0])
                print("truth:", truth[0:72,0])
                

                for i, j in zip(note_idx, note_idx[1:]):
                   print(prediction[i:j,0])




                

In [233]:
evaluate_accuracy_for_one(model,train_dataloader,part_dic)

note_idx: [72.0, 78.0, 84.0, 90.0, 99.0, 100.5, 102.0, 108.0, 114.0, 120.0, 129.0, 132.0, 135.0, 138.0, 141.0, 144.0, 147.0, 150.0, 153.0, 156.0, 159.0, 162.0, 165.0, 168.0, 171.0, 174.0, 180.0, 186.0, 192.0, 198.0, 201.0, 204.0, 207.0, 210.0, 219.0, 222.0, 225.0, 228.0, 231.0, 234.0, 237.0, 240.0, 243.0, 246.0, 270.0, 282.0, 294.0, 300.0, 306.0, 315.0, 316.5, 318.0, 324.0, 330.0, 336.0, 345.0, 348.0, 351.0, 354.0, 360.0, 369.0, 372.0, 375.0, 378.0, 402.0, 408.0, 414.0, 438.0, 450.0, 453.0, 456.0, 459.0, 462.0, 465.0, 468.0, 471.0, 474.0, 477.0, 480.0, 483.0, 486.0, 492.0, 504.0, 507.0, 510.0, 516.0, 522.0, 528.0, 531.0, 534.0, 537.0, 540.0, 543.0, 546.0, 549.0, 552.0, 555.0, 555.0, 558.0, 558.0, 564.0, 567.0, 570.0, 576.0, 582.0, 588.0, 600.0, 603.0, 606.0, 615.0, 618.0, 630.0, 702.0, 708.0, 714.0, 723.0, 724.5, 726.0, 732.0, 738.0, 744.0, 750.0, 759.0, 760.5, 762.0, 768.0, 774.0, 780.0, 789.0, 792.0, 795.0, 798.0, 804.0, 810.0, 816.0, 822.0, 828.0, 831.0, 834.0, 837.0, 840.0, 843.0, 

TypeError: ignored

In [201]:
(105.5+2)*12

1290.0

start time, duration , pitch to separate 

use the onset_beat and duration_beat

multiply them according to the values set when producing the pianorolls 

-> get the position in the pianoroll

time_div = 12



# Visualization

In [ ]:
plt.plot(his["val_accuracy_v0"],'-o')
plt.plot(his["val_accuracy_v1"],'-o')
plt.plot(his["val_accuracy_v2"],'-o')
plt.plot(his["val_accuracy_v3"],'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Accuracy0','Accuracy1','Accuracy2','Accuracy3'])
plt.title('Accuracy vs Epochs')

plt.show()

In [ ]:
plt.plot(his["val_accuracy"],'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend('Accuracy')
plt.title('Accuracy vs Epochs')

plt.show()

In [ ]:
plt.plot(his["train_loss"],'-o')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Loss'])
plt.title('Loss vs Epochs')

plt.show()

# Old training loop - matrix and non matrix format

def training_loop(model,optimizer, train_dataloader, monophonic, epochs=50, val_dataloader=None, device=None, scheduler=None):
    if device is None:
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f"Training on device: {device}")

    print("monophonic set to:",monophonic)
    model = model.to(device)
    history = defaultdict(list)

    for i_epoch in range(1, epochs + 1):
        loss_sum = 0
        #accuracy_v0_sum = 0
        #accuracy_v1_sum = 0
        #accuracy_v2_sum = 0
        #accuracy_v3_sum = 0

        accuracy_sum_list = [0 for i in range(5)]                               ########## FIXED FOR 5 voices MAX right now - b.c. FUGUES have max 5 voices
        val_accuracy_sum_list = [0 for i in range(5)]                               ########## FIXED FOR 5 voices MAX right now - b.c. FUGUES have max 5 voices

        accuracy_v_all_sum = 0
        model.train()
        accuracy_sum = 0
        

        for idx, (voices, lens, nbr_voices) in enumerate(train_dataloader):  
            
            voices = voices.to(device).float()
            optimizer.zero_grad()
            loss = model.forward(voices, lens, nbr_voices)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()    

            if monophonic == False:
                with torch.no_grad():
                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)                      

                    v_pred_argm = torch.tensor(np.argmax(prediction,axis=0))
                    mask_pred = (prediction.sum(axis=0) == 0)
                    v_pred_argm[mask_pred] = -1
                    v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)

                    
                    single_voices = voices[:,:,:,:-1]

                    v_ori_argm = torch.argmax(np.squeeze(single_voices,axis=0).cpu(),axis=2)
                    mask_ori = ((np.squeeze(single_voices,axis=0).cpu()).sum(axis=2) == 0).numpy()
                    v_ori_argm[mask_ori] = -1
                    v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)
                    acc = accuracy_score(v_pred_flat,v_ori_flat)  
                    accuracy_sum += acc 

                    """
                    if nbr_voices == 4: 
                        v_pred_comb = torch.stack([torch.tensor(pred_v0), torch.tensor(pred_v1), torch.tensor(pred_v2), torch.tensor(pred_v3)], dim=2)
                    if nbr_voices ==3:
                        v_pred_comb = torch.stack([torch.tensor(pred_v0), torch.tensor(pred_v1), torch.tensor(pred_v2)], dim=2)
                    v_pred_argm = v_pred_comb.argmax(dim=2)
                    mask_pred = (v_pred_comb.sum(axis=2) == 0).numpy()
                    v_pred_argm[mask_pred] = -1
                    v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)
                    print("v_pred_flat:", v_pred_flat.shape)
                    """
                    """
                    if nbr_voices == 4:                   
                        v_ori_comb = torch.stack([np.squeeze(voices[:,:,:,0]).cpu(), np.squeeze(voices[:,:,:,1]).cpu(), np.squeeze(voices[:,:,:,2]).cpu(), np.squeeze(voices[:,:,:,3]).cpu()], dim=2)
                    if nbr_voices ==3:
                        v_ori_comb = torch.stack([np.squeeze(voices[:,:,:,0]).cpu(), np.squeeze(voices[:,:,:,1]).cpu(), np.squeeze(voices[:,:,:,2]).cpu()], dim=2)
                    v_ori_argm = v_ori_comb.argmax(dim=2)
                    mask_ori = (v_ori_comb.sum(axis=2) == 0).numpy()
                    print("old mask", mask_ori.shape)
                    v_ori_argm[mask_ori] = -1
                    v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)
                    print("v_ori_flat", v_ori_flat.shape)
                    acc = accuracy_score(v_pred_flat,v_ori_flat)   
                    print("acc",acc)                    
                    accuracy_sum += acc 
                    """


            if monophonic == True:
                with torch.no_grad():

                    prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                    prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)

                    truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()

                    acc_list = [0 for i in range(len(prediction[0,:]))]

                    for i in range(len(prediction[0,:])):
                      acc_list[i] = accuracy_score(prediction[:,i], truth[:,i])
                      accuracy_sum_list[i] += acc_list[i]/len(lens)

                    
                    """
                    pred_v0, pred_v1, pred_v2, pred_v3 = model.predict(voices[:,:,:,-1], lens,monophonic)

                    acc_v0 = accuracy_score(torch.tensor(pred_v0), np.squeeze(voices[:,:,:,0]).argmax(dim=1).cpu())
                    acc_v1 = accuracy_score(torch.tensor(pred_v1), np.squeeze(voices[:,:,:,1]).argmax(dim=1).cpu())
                    acc_v2 = accuracy_score(torch.tensor(pred_v2), np.squeeze(voices[:,:,:,2]).argmax(dim=1).cpu())
                    if nbr_voices == 4:
                        acc_v3 = accuracy_score(torch.tensor(pred_v3), np.squeeze(voices[:,:,:,3]).argmax(dim=1).cpu())
                    
                    # normalize according to the number of sequences in the batch (atm len(lens)==1)
                    accuracy_v0_sum += acc_v0 / len(lens)
                    accuracy_v1_sum += acc_v1 / len(lens)
                    accuracy_v2_sum += acc_v2 / len(lens)
                    if nbr_voices == 4:
                        accuracy_v3_sum += acc_v3 / len(lens)
                    """



        train_loss = loss_sum / len(train_dataloader)

        # normalize according to the number of batches
        if monophonic == True:

            train_acc_list = np.array(accuracy_sum_list) / len(train_dataloader)
            train_acc_list[3] = accuracy_sum_list[3] / 18                       ## bc only 18 pieces with len 3
            train_acc_list[4] = accuracy_sum_list[4] / 2                         ##### CHECK IF 2 or 3 pieces with len 4


            history["train_loss"].append(train_loss)
            history["train_acc"].append(train_acc_list)
            print("Train Loss: {}, Train Accuracy_0 : {}, Train Accuracy_1 : {},Train Accuracy_2 : {}, Train Accuracy_3 : {}, Train Accuracy_4 : {}".format(train_loss, train_acc_list[0], train_acc_list[1], train_acc_list[2], train_acc_list[3],train_acc_list[4])) 


            """
            train_accuracy_v0 = accuracy_v0_sum / len(train_dataloader)
            train_accuracy_v1 = accuracy_v1_sum / len(train_dataloader)
            train_accuracy_v2 = accuracy_v2_sum / len(train_dataloader)
            train_accuracy_v3 = accuracy_v3_sum / 18   ## bc only 18 pieces with len 3

            history["train_loss"].append(train_loss)
            history["train_accuracy_v0"].append(train_accuracy_v0)
            history["train_accuracy_v1"].append(train_accuracy_v1)
            history["train_accuracy_v2"].append(train_accuracy_v2)
            #if nbr_voices == 4:
            history["train_accuracy_v3"].append(train_accuracy_v3)
            print("Train Loss: {}, Train Accuracy_0 : {}, Train Accuracy_1 : {},Train Accuracy_2 : {}, Train Accuracy_3 : {}".format(train_loss, train_accuracy_v0, train_accuracy_v1, train_accuracy_v2, train_accuracy_v3)) 
            #else:
            #    print("Train Loss: {}, Train Accuracy_0 : {}, Train Accuracy_1 : {},Train Accuracy_2 : {}".format(train_loss, train_accuracy_v0, train_accuracy_v1, train_accuracy_v2)) 
            """

        if monophonic == False:
            train_accuracy = accuracy_sum / len(train_dataloader)

            history["train_loss"].append(train_loss)
            history["train_accuracy"].append(train_accuracy)
            print("Train Loss: {}, Train Accuracy : {}".format(train_loss, train_accuracy)) 


        if monophonic == True:
            if val_dataloader is not None:
                # Evaluate on the validation set
                model.eval()
                accuracy_v0_sum = 0
                accuracy_v1_sum = 0
                accuracy_v2_sum = 0
                accuracy_v3_sum = 0

                with torch.no_grad():

                    for voices, lens, nbr_voices in val_dataloader:

                        voices = voices.to(device).float()

                        prediction = model.predict(voices[:,:,:,-1], lens, monophonic)  
                        prediction = torch.swapaxes(torch.tensor(prediction), 0, 1)

                        truth = np.squeeze(voices[:,:,:,:-1]).argmax(dim=1).cpu()

                        acc_list = [0 for i in range(len(prediction[0,:]))]

                        for i in range(len(prediction[0,:])):
                          acc_list[i] = accuracy_score(prediction[:,i], truth[:,i])
                          val_accuracy_sum_list[i] += acc_list[i]/len(lens)


                        #print("val_accuracy_sum_list[3]",val_accuracy_sum_list[3])
                    #val_acc_list = np.array(val_accuracy_sum_list) / len(train_dataloader)
                    #val_acc_list[3] = val_acc_list[3] / 18                       ## bc only 18 pieces with len 3
                    #val_acc_list[4] = val_acc_list[4] / 2                         ##### CHECK IF 2 or 3 pieces with len 4


                #history["val_acc_new"].append(val_acc_list)
                #print(" Validation Accuracy_0 : {}, Validation Accuracy_1 : {}, Validation Accuracy_2 : {}, Validation Accuracy_3 : {}, Validation Accuracy_4 : {}".format(val_acc_list[0], val_acc_list[1], val_acc_list[2], val_acc_list[3],val_acc_list[4]))



                        # Predict the model's output on a batch
                        pred_v0, pred_v1, pred_v2, pred_v3 = model.predict(voices[:,:,:,-1], lens,monophonic)
                            
                        # compute the accuracy 
                        acc_v0 = accuracy_score(torch.tensor(pred_v0), np.squeeze(voices[:,:,:,0]).argmax(dim=1).cpu())
                        acc_v1 = accuracy_score(torch.tensor(pred_v1), np.squeeze(voices[:,:,:,1]).argmax(dim=1).cpu())
                        acc_v2 = accuracy_score(torch.tensor(pred_v2), np.squeeze(voices[:,:,:,2]).argmax(dim=1).cpu())
                        if nbr_voices == 4:
                            acc_v3 = accuracy_score(torch.tensor(pred_v3), np.squeeze(voices[:,:,:,3]).argmax(dim=1).cpu())
                            
                            
                        # normalize according to the number of sequences in the batch (atm len(lens)==1)
                        accuracy_v0_sum += acc_v0 / len(lens)
                        accuracy_v1_sum += acc_v1 / len(lens)
                        accuracy_v2_sum += acc_v2 / len(lens)
                        if nbr_voices == 4:
                            accuracy_v3_sum += acc_v3 / len(lens)

                    # normalize according to the number of batches
                    val_accuracy_v0 = accuracy_v0_sum / len(val_dataloader)
                    val_accuracy_v1 = accuracy_v1_sum / len(val_dataloader)
                    val_accuracy_v2 = accuracy_v2_sum / len(val_dataloader)
                    val_accuracy_v3 = accuracy_v3_sum / 18  ##len(val_dataloader). - bc 18 pieces only with voice 3


                    val_acc_list = np.array(val_accuracy_sum_list) / len(val_dataloader)
                    val_acc_list[3] = val_accuracy_sum_list[3] / 18                       ## bc only 18 pieces with len 3
                    val_acc_list[4] = val_accuracy_sum_list[4] / 2                         ##### CHECK IF 2 or 3 pieces with len 4
                    


                history["val_accuracy_v0"].append(val_accuracy_v0)
                history["val_accuracy_v1"].append(val_accuracy_v1)
                history["val_accuracy_v2"].append(val_accuracy_v2)
                #if nbr_voices == 4:
                history["val_accuracy_v3"].append(val_accuracy_v3)
                print(" Validation Accuracy_0 : {}, Validation Accuracy_1 : {}, Validation Accuracy_2 : {}, Validation Accuracy_3 : {}".format(val_accuracy_v0, val_accuracy_v1, val_accuracy_v2, val_accuracy_v3))
                #else:
                #    print(" Validation Accuracy_0 : {}, Validation Accuracy_1 : {}, Validation Accuracy_2 : {}".format(val_accuracy_v0, val_accuracy_v1, val_accuracy_v2))


                history["val_acc_new"].append(val_acc_list)
                print(" Validation Accuracy_0 : {}, Validation Accuracy_1 : {}, Validation Accuracy_2 : {}, Validation Accuracy_3 : {}, Validation Accuracy_4 : {}".format(val_acc_list[0], val_acc_list[1], val_acc_list[2], val_acc_list[3],val_acc_list[4]))


        if monophonic == False:
            if val_dataloader is not None:
                # Evaluate on the validation set
                model.eval()
                accuracy_sum = 0
                
                with torch.no_grad():
                    for voices, lens, nbr_voices in val_dataloader:

                        voices = voices.to(device).float()
                        
                        # Predict the model's output on a batch

                        prediction = model.predict(voices[:,:,:,-1], lens, monophonic)                      

                        v_pred_argm = torch.tensor(np.argmax(prediction,axis=0))
                        mask_pred = (prediction.sum(axis=0) == 0)
                        v_pred_argm[mask_pred] = -1
                        v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)
                        
                        single_voices = voices[:,:,:,:-1]

                        v_ori_argm = torch.argmax(np.squeeze(single_voices,axis=0).cpu(),axis=2)
                        mask_ori = ((np.squeeze(single_voices,axis=0).cpu()).sum(axis=2) == 0).numpy()
                        v_ori_argm[mask_ori] = -1
                        v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)
                        acc = accuracy_score(v_pred_flat,v_ori_flat)  
                        accuracy_sum += acc 

                        """
                        pred_v0, pred_v1, pred_v2, pred_v3 = model.predict(voices[:,:,:,-1], lens,monophonic)
                        if nbr_voices == 4: 
                            v_pred_comb = torch.stack([torch.tensor(pred_v0), torch.tensor(pred_v1), torch.tensor(pred_v2), torch.tensor(pred_v3)], dim=2)
                        if nbr_voices ==3:
                            v_pred_comb = torch.stack([torch.tensor(pred_v0), torch.tensor(pred_v1), torch.tensor(pred_v2)], dim=2)
                        v_pred_argm = v_pred_comb.argmax(dim=2)
                        mask_pred = (v_pred_comb.sum(axis=2) == 0).numpy()
                        v_pred_argm[mask_pred] = -1
                        v_pred_flat = torch.flatten(v_pred_argm, start_dim=0, end_dim=-1)                      
                        if nbr_voices == 4:                   
                            v_ori_comb = torch.stack([np.squeeze(voices[:,:,:,0]).cpu(), np.squeeze(voices[:,:,:,1]).cpu(), np.squeeze(voices[:,:,:,2]).cpu(), np.squeeze(voices[:,:,:,3]).cpu()], dim=2)
                        if nbr_voices ==3:
                            v_ori_comb = torch.stack([np.squeeze(voices[:,:,:,0]).cpu(), np.squeeze(voices[:,:,:,1]).cpu(), np.squeeze(voices[:,:,:,2]).cpu()], dim=2)
                        v_ori_argm = v_ori_comb.argmax(dim=2)
                        mask_ori = (v_ori_comb.sum(axis=2) == 0).numpy()
                        v_ori_argm[mask_ori] = -1
                        v_ori_flat = torch.flatten(v_ori_argm, start_dim=0, end_dim=-1)

                        acc = accuracy_score(v_pred_flat,v_ori_flat)
                        accuracy_sum += acc 
                        """
                        
                    # normalize according to the number of batches
                    val_accuracy = accuracy_sum / len(val_dataloader)

                history["val_accuracy"].append(val_accuracy)  
                print(" Validation Accuracy : {}".format(val_accuracy))


        if scheduler is not None:
            scheduler.step()

        # save the model
        torch.save(model, Path("./AI-MA_project/model_temp_epoch{}.pkl".format(i_epoch)))

    return history